In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import matplotlib.colors as colors

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace
import lib.datagen as datagen

from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf
from tensorflow import keras
import numpy as np

model_name = "deconvolve_gauss_grid_oneloc"
model = keras.models.load_model(f'trained_networks/{model_name}.h5')
#img = image.imread("example_image.png").sum(axis=2)
distorted_img_syn, img_syn = datagen.get_synthetic_data_pair()

distorted_img_9 = np.load('example_files/test_data_9.npy')/1e6
distorted_img_0 = np.load('example_files/test_data_0.npy')/1e6

recovered_img_syn = model(distorted_img_syn[np.newaxis,:,:], training=False).numpy()[0,:,:,0]
recovered_img_9 = model(distorted_img_9[np.newaxis,:,:], training=False).numpy()[0,:,:,0]
recovered_img_0 =  model(distorted_img_0[np.newaxis,:,:], training=False).numpy()[0,:,:,0]

percent = 99
perc = np.percentile(distorted_img_0, percent)

# fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)

# axs[0].imshow(img_syn, vmax=perc, vmin=0)
# axs[0].set_title("Synthetic Ground Thruth")
# axs[1].imshow(distorted_img_syn, vmax=perc, vmin=0)
# axs[1].set_title("Synthetic Data")
# axs[2].imshow(recovered_img_syn, vmax=perc, vmin=0)
# axs[2].set_title("Recovered Synthetic Data")

# fig, axs = plt.subplots(2, 2, figsize=(5, 10), sharex=True, sharey=True)
# axs[0, 0].imshow(distorted_img_0, vmax=perc, vmin=0)
# axs[0, 0].set_title("Blurred Captures")
# axs[0, 1].imshow(recovered_img_0, vmax=perc, vmin=0)
# axs[0, 1].set_title("Recovered deepBlind Images")
# axs[1, 0].imshow(distorted_img_9, vmax=perc, vmin=0)
# axs[1, 1].imshow(recovered_img_9, vmax=perc, vmin=0)

# plt.show()

G = xtrace.depth_spill_psf(datagen.global_config, *utils.ray_grid(datagen.global_config["dimensions"]))
recovered_img_conv_0 = deconvolution.richard_lucy(distorted_img_0, G, 6)

fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
axs[0].imshow(distorted_img_0, vmax=perc, vmin=0)
axs[0].set_title("Detector Image")
axs[1].imshow(recovered_img_conv_0, vmax=perc, vmin=0)
axs[1].set_title("Richard Lucy Deconvolution")
axs[2].imshow(recovered_img_0, vmax=perc, vmin=0)
axs[2].set_title("ML deconvolution")

# fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
# axs[0].imshow(img_syn, vmax=perc, vmin=0)
# axs[0].set_title("Synthetic Ground Thruth")
# axs[1].imshow(distorted_img_syn, vmax=perc, vmin=0)
# axs[1].set_title("Synthetic Data")
# axs[2].imshow(distorted_img_0, vmax=perc, vmin=0)
# axs[2].set_title("Real Data")
# plt.show()

distorted_conv = (G.get()@recovered_img_conv_0.flatten()).reshape(recovered_img_conv_0.shape)
distorted_ml = (G.get()@recovered_img_0.flatten()).reshape(recovered_img_0.shape)


vmax= np.max(np.maximum(distorted_img_0, distorted_ml, distorted_conv))
norm = colors.PowerNorm(gamma=0.5, vmin=0, vmax=vmax)
fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
pcm = axs[0].imshow(distorted_img_0, norm=norm)
axs[0].set_title("Detector Image")
axs[1].imshow(distorted_conv, norm=norm)
axs[1].set_title("Richard Lucy, Convoluted")
axs[2].imshow(distorted_ml, norm=norm)
axs[2].set_title("ML, Convoluted")
fig.colorbar(pcm, ax=axs, shrink=1, location='bottom')
fig.suptitle("Visual Comparison of Inversion")

diff_rl = np.abs(distorted_img_0 - distorted_conv)
#diff_rl = np.sign(diff_rl)*np.sqrt(np.abs(diff_rl))
diff_ml = np.abs(distorted_img_0 - distorted_ml)
#diff_ml = np.sign(diff_ml)*np.sqrt(np.abs(diff_ml))

vmax = np.max(np.maximum(diff_rl, diff_ml))
norm = colors.PowerNorm(gamma=0.5, vmin=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True, constrained_layout=True)
#pcm_1 = axs[0].imshow(distorted_img_0, vmax=perc, vmin=0)
pcm = axs[0].imshow(diff_rl, norm=norm)
axs[0].set_title("Difference (original - RL)")
axs[1].imshow(diff_ml, norm=norm)
axs[1].set_title("Difference (original - ML)")
fig.colorbar(pcm, ax=axs, shrink=1, location='bottom')
plt.show()

plt.figure()
plt.imshow(distorted_img_0)
plt.title("Original Image")
plt.colorbar()
#Add local energy loss constraint? (ex: 10x10 area should contain about the same ammount of energy)
#Add positivity loss constraint?
#Try the Pecent contraint with lower randomness value -> trains on more realistic schenarios

bshuf filter already loaded, skip it.
2022-04-19 09:29:49.069769: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-19 09:29:49.075164: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-04-19 09:29:49.075204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12143 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:18:00.0, compute capability: 7.0
2022-04-19 09:29:53.294638: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …